<h1><b><center>POC: Solução de Análise Automatizada dos Recursos de 3ª Instância dos Pedidos de Acesso à Informação</center></b></h1>

**Sobre este Notebook**: Prova de conceito e criação de pipeline do projeto de desenvolvimento da solução de análise automatizada dos recursos de 3ª instância dos pedidos de acesso a informação impetrados à CGU.

**Autor**: Douglas Rolins de Santana / douglasrolins@discente.ufg.br

**Sobre o Projeto:**

A Lei Federal nº 12.527/2011 regulamenta os procedimentos e fluxos para o acesso as informações dos órgãos públicos do Brasil. No âmbito do Poder Executivo Federal é utilizado a plataforma Fala.BR como ponto de contato entre os cidadãos e os orgãos. Esta plataforma é mantida pela Controladoria Geral da União (CGU), que também é responsável por responder os pedidos de recursos em 3ª instância.

Qualquer pessoa pode registrar um pedido de informação a um orgão pelo Fala.BR, sendo respondido pela mesma plataforma. Caso o pedido seja negado ou incompleto, o requerente poderá interpor recurso contra a decisão, recurso este que será analisado pela autoridade hierarquicamente superior à que exarou a decisão impugnada (1ª instância). No caso de nova negativa, o requerente poderá abrir novo recurso que será direcionado a autoridade máxima do orgão (2º instância). No caso de nova negativa, o requerente poderá interpor recurso a Controladoria Geral da União que fará a avaliação (3ª instância).

A CGU possui equipe própria para analisar e responder os recursos de 3º instância de todo o Brasil. Na data atual (out/2023) há aproximadamente 1.300.000 pedidos de informação registrados, 160.000 recursos, considerando todas as instâncias, e 17.000 recursos de 3ª instância.

Diante a alta demanda de recursos de 3ª instância e para auxiliar a equipe no tratamentos desses recursos, propõe-se uma ferramenta que automatize o reconhecimento de recursos semelhantes a fim de auxiliar na resposta a um novo recurso.

Hipótese inicial: A partir dos dados de recursos já analisados e com uma amostra de dados anotados(indicação de recursos e pedidos semelhantes pela área compenente), desenvolver modelo de linguagem que gere as representações dos recursos para posteriormente verificação da similaridade. Utilizar essas recomendações para gerar texto de resposta com um grande modelo de linguagem.

# 0 - Documentação Inicial e Bases de Dados

Possíveis soluções:
1. Gerar recomendações de pedidos e recursos similares para auxiliar na resposta de um novo recurso;
2. Gerar recomendação de decisão para o recurso (deferido/indeferido/etc) a partir dos recursos similares já respondidos;
3. Gerar texto de resposta do recurso.


Pipeline básico inicial para Recomendação:
1. Treinamento inicial do modelo de linguagem que gera as representações vetoriais
2. Geração das representações de cada pedido/recurso da base de dados
3. Geração da representação vetorias do novo pedido/recurso
4. Verificar similaridade entre o recurso novo com os top-k recursos da base

Pipeline em ambiente de produção:
1. Modelo de linguagem hospedado e acessado via API
2. Base de dados vetorial para armazenar a representação de cada recurso
3. A medida que cada recurso é cadastrado, gera a representação vetorial do mesmo
4. Interface de acesso pelo usuário que pode selecionar um recurso e o sistema retornar os k recursos similares
Extra: O usuário poderá marcar como "não similar" algum item do resultados dos k recursos similares e essa informação servir para novo treinamento do modelo, que pode ser feito periodicamente, gerando novo modelo a ser utilizado na fase de representação vetorial de cada recurso recém cadastrado, ou mesmo gerar nova representação a todos os recursos novamente.

Bases de Dados:

* Dados públicos dos pedidos de acesso a informação de 2015 a 2023: https://buscalai.cgu.gov.br/DownloadDados/DownloadDados
* Dicionário de Dados: https://buscalai.cgu.gov.br/files/Dicionario-Dados-Exportacao.txt

------------------------------
------------------------------
#### CAMPOS: PEDIDOS
------------------------------
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido;
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- ResumoSolicitacao - texto(255): resumo do pedido;
- DetalhamentoSolicitacao - texto(2048): detalhamento do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Resposta - texto(8000): resposta ao pedido;
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");

------------------------------
------------------------------
#### CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- DescRecurso - texto(8000): descrição do recurso;
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- OrgaoPedido - texto(250): nome do órgão destinatário do pedido;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- RespostaRecurso - texto(8000): resposta ao recurso;
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

------------------------------
------------------------------
#### CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;
------------------------------
------------------------------

# 1 - Carregar os Dados

In [ ]:
# BIBLIOTECAS
import os
import chardet
import pandas as pd

In [ ]:
# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Acessar arquivos CSV
versao_download = 'download_jan_2024'


path = "/content/drive/MyDrive/Colab Notebooks/dataset_cgu/"+versao_download
lista_ano = ['2015','2016','2017','2018','2019','2020','2021','2022','2023',]
folder_mask = "Arquivos_csv_"
files_pedidos = {'2015':'20240124_Pedidos_csv_2015.csv',
                 '2016':'20240124_Pedidos_csv_2016.csv',
                 '2017':'20240124_Pedidos_csv_2017.csv',
                 '2018':'20240124_Pedidos_csv_2018.csv',
                 '2019':'20240124_Pedidos_csv_2019.csv',
                 '2020':'20240124_Pedidos_csv_2020.csv',
                 '2021':'20240124_Pedidos_csv_2021.csv',
                 '2022':'20240123_Pedidos_csv_2022.csv',
                 '2023':'20240123_Pedidos_csv_2023.csv'}

files_recursos = {'2015':'20240124_Recursos_csv_2015.csv',
                 '2016':'20240124_Recursos_csv_2016.csv',
                 '2017':'20240124_Recursos_csv_2017.csv',
                 '2018':'20240124_Recursos_csv_2018.csv',
                 '2019':'20240124_Recursos_csv_2019.csv',
                 '2020':'20240124_Recursos_csv_2020.csv',
                 '2021':'20240124_Recursos_csv_2021.csv',
                 '2022':'20240123_Recursos_csv_2022.csv',
                 '2023':'20240123_Recursos_csv_2023.csv'}

files_solicitantes_pedidos = {'2015':'20240124_SolicitantesPedidos_csv_2015.csv',
                            '2016':'20240124_SolicitantesPedidos_csv_2016.csv',
                            '2017':'20240124_SolicitantesPedidos_csv_2017.csv',
                            '2018':'20240124_SolicitantesPedidos_csv_2018.csv',
                            '2019':'20240124_SolicitantesPedidos_csv_2019.csv',
                            '2020':'20240124_SolicitantesPedidos_csv_2020.csv',
                            '2021':'20240124_SolicitantesPedidos_csv_2021.csv',
                            '2022':'20240123_SolicitantesPedidos_csv_2022.csv',
                            '2023':'20240123_SolicitantesPedidos_csv_2023.csv',}

files_solicitantes_recursos = {'2015':'20240124_SolicitantesRecursos_csv_2015.csv',
                          '2016':'20240124_SolicitantesRecursos_csv_2016.csv',
                          '2017':'20240124_SolicitantesRecursos_csv_2017.csv',
                          '2018':'20240124_SolicitantesRecursos_csv_2018.csv',
                          '2019':'20240124_SolicitantesRecursos_csv_2019.csv',
                          '2020':'20240124_SolicitantesRecursos_csv_2020.csv',
                          '2021':'20240124_SolicitantesRecursos_csv_2021.csv',
                          '2022':'20240123_SolicitantesRecursos_csv_2022.csv',
                          '2023':'20240123_SolicitantesRecursos_csv_2023.csv'}

def busca_codificacao(file_name):
  with open(file_name, 'rb') as file:
    result = chardet.detect(file.read())
  return result['encoding']

def carrega_dados(ano=None):
    encoding = 'UTF-16'

    if ano:
        dataset_pedidos = pd.read_csv(f"{path}/{folder_mask}{ano}/{files_pedidos[ano]}", delimiter=';', encoding=encoding)
        dataset_recursos = pd.read_csv(f"{path}/{folder_mask}{ano}/{files_recursos[ano]}", delimiter=';', encoding=encoding)
        dataset_sol_ped = pd.read_csv(f"{path}/{folder_mask}{ano}/{files_solicitantes_pedidos[ano]}", delimiter=';', encoding=encoding)
        dataset_sol_rec = pd.read_csv(f"{path}/{folder_mask}{ano}/{files_solicitantes_recursos[ano]}", delimiter=';', encoding=encoding)
    elif ano is None:
        datasets_pedidos = [pd.read_csv(f"{path}/{folder_mask}{ano}/{files_pedidos[ano]}", delimiter=';', encoding=encoding) for ano in lista_ano]
        datasets_recursos = [pd.read_csv(f"{path}/{folder_mask}{ano}/{files_recursos[ano]}", delimiter=';', encoding=encoding) for ano in lista_ano]
        datasets_sol_ped = [pd.read_csv(f"{path}/{folder_mask}{ano}/{files_solicitantes_pedidos[ano]}", delimiter=';', encoding=encoding) for ano in lista_ano]
        datasets_sol_rec = [pd.read_csv(f"{path}/{folder_mask}{ano}/{files_solicitantes_recursos[ano]}", delimiter=';', encoding=encoding) for ano in lista_ano]

        dataset_pedidos = pd.concat(datasets_pedidos, ignore_index=True)
        dataset_recursos = pd.concat(datasets_recursos, ignore_index=True)
        dataset_sol_ped = pd.concat(datasets_sol_ped, ignore_index=True)
        dataset_sol_rec = pd.concat(datasets_sol_rec, ignore_index=True)
    else:
        return None

    return dataset_pedidos, dataset_recursos, dataset_sol_ped, dataset_sol_rec

In [ ]:
# Carregar Dados para os respectivos dataframes
#dt_pedidos, dt_recursos, dt_sol_ped, dt_sol_rec = carrega_dados(ano='2016')
dt_pedidos, dt_recursos, dt_sol_ped, dt_sol_rec = carrega_dados(ano=None)

# 2 - Amostra e Análise dos Dados

In [ ]:
print("Total de Pedidos:",len(dt_pedidos))
print("Total de Recursos:",len(dt_recursos))
print("Total de Solicitantes de Pedidos",len(dt_sol_ped))
print("Total de Solicitantes de Recursos",len(dt_sol_rec))

Total de Pedidos: 780084
Total de Recursos: 82124
Total de Solicitantes de Pedidos 328953
Total de Solicitantes de Recursos 26698


In [ ]:
dt_pedidos['ResumoSolicitacao'].iloc[159]

'LIBERAÇÃO DE RECURSOS PARA CONSTRUÇÃO DA UPA/JAPERI/RJ'

In [ ]:
dt_recursos[dt_recursos['Instancia'] == 'CGU']

,IdRecurso,IdRecursoPrecedente,DescRecurso,IdPedido,IdSolicitante,ProtocoloPedido,OrgaoPedido,OrgaoDestinatario,Instancia,Situacao,DataRegistro,PrazoAtendimento,OrigemSolicitacao,TipoRecurso,DataResposta,RespostaRecurso,TipoResposta
13,100066,98643,Bom dia. O meu pedido de informação foi realiz...,2114809,2503254,2680000517201515,IBAMA – Instituto Brasileiro do Meio Ambiente ...,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,06/05/2015,11/05/2015,Internet,Resposta não foi dada no prazo,11/05/2015,DECISÃO No exercício das atribuições a mim co...,Deferido
38,100204,98485,• Qual a nota atribuída ao item 5 do plano de ...,2077340,2511912,23480002787201589,UFAM – Fundação Universidade do Amazonas,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,07/05/2015,12/05/2015,Internet,Informação incompleta,29/06/2015,DECISÃO No exercício das atribuições a mi...,Indeferido
43,100226,98983,No dia 20/03/2015 eu fiz a solicitação da cópi...,2130157,2140825,8850000945201517,MJSP – Ministério da Justiça e Segurança Pública,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,07/05/2015,12/05/2015,Internet,Ausência de justificativa legal para classific...,19/06/2015,DECISÃO No exercício das atribuições a mim...,Não conhecimento
49,100251,96833,O DPF e o MJ entendem que a pretensão de acess...,2114114,2105340,8850000829201506,PF – Polícia Federal,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,07/05/2015,12/05/2015,Internet,Informação incompleta,29/10/2015,DECISÃO No exercício das atribuições a mim...,Indeferido
57,100282,98923,Reitero os pedidos inicias. Peço deferimento.,2157543,1319974,3950000642201552,ME - Ministério da Economia,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,07/05/2015,15/05/2015,Internet,Justificativa para o sigilo insatisfatória/não...,24/06/2015,DECISÃO No exercício das atribuições a mim...,Indeferido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99741,187804,186910,"Com fundamento no Art. 16º da Lei 12.527, de 1...",6146454,5264869,23546070974202393,"IFCE – Instituto Federal de Educação, Ciência ...",CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,29/09/2023,01/11/2023,Internet,Outros,10/10/2023,D E C I S Ã O No exercício das atribuições...,Não conhecimento
99798,187884,187649,"Se o endereço eletrônico é da Casa Civil, é a ...",6048164,1803907,137013388202368,CC-PR – Casa Civil da Presidência da República,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,02/10/2023,09/10/2023,Internet,Informação incompleta,05/10/2023,D E C I S Ã O No uso das competências previ...,Não conhecimento
99800,187888,187590,Primeiramente quem está respondendo não está i...,6227181,1318830,23546077269202317,UTFPR – Universidade Tecnológica Federal do Pa...,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,02/10/2023,01/11/2023,Internet,Informação incompleta,17/10/2023,D E C I S Ã O No exercício das atribuições a ...,Não conhecimento
99833,187953,187940,"Prezados, Tem sido recorrente solicitar infor...",6331370,5066636,23546085090202333,INEP – Instituto Nacional de Estudos e Pesquis...,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,03/10/2023,09/10/2023,Internet,Informação incompleta,05/10/2023,D E C I S Ã O No uso das competências previst...,Não conhecimento


In [ ]:
dt_pedidos[dt_pedidos['IdPedido'] == 3878674]

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
0,3878674,60141000001202295,Federal,COMAER – Comando da Aeronáutica,Concluída,01/01/2022,Porque a FAB é chamada de Comando da Aeronaút...,Porque a FAB é chamada de Comando da Aeronaút...,03/02/2022,Sim,...,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,02/02/2022,MINISTÉRIO DA DEFESA COMANDO DA AERONÁUTICA ...,Não se trata de solicitação de informação,


In [ ]:
dt_recursos[dt_recursos['IdRecurso'] == 100204]

,IdRecurso,IdRecursoPrecedente,DescRecurso,IdPedido,IdSolicitante,ProtocoloPedido,OrgaoPedido,OrgaoDestinatario,Instancia,Situacao,DataRegistro,PrazoAtendimento,OrigemSolicitacao,TipoRecurso,DataResposta,RespostaRecurso,TipoResposta
38,100204,98485,• Qual a nota atribuída ao item 5 do plano de ...,2077340,2511912,23480002787201589,UFAM – Fundação Universidade do Amazonas,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,07/05/2015,12/05/2015,Internet,Informação incompleta,29/06/2015,DECISÃO No exercício das atribuições a mi...,Indeferido


In [ ]:
dt_recursos[dt_recursos['IdRecurso'] == 98485]

,IdRecurso,IdRecursoPrecedente,DescRecurso,IdPedido,IdSolicitante,ProtocoloPedido,OrgaoPedido,OrgaoDestinatario,Instancia,Situacao,DataRegistro,PrazoAtendimento,OrigemSolicitacao,TipoRecurso,DataResposta,RespostaRecurso,TipoResposta


In [ ]:
dt_pedidos[dt_pedidos['IdPedido'] == 98485]

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao


In [ ]:
# Verificando por pedidos duplicados na coluna 'ProtocoloPedido'
pedidos_duplicados = dt_pedidos['ProtocoloPedido'].duplicated()

# Verifica se há algum valor duplicado
tem_pedidos_duplicados = pedidos_duplicados.any()

if tem_pedidos_duplicados:
    print("Existem pedidos duplicados na coluna ProtocoloPedido.")
    # Opcional: para ver quais são os valores duplicados
    valores_duplicados = dt_pedidos['NumeroProtocolo'][pedidos_duplicados]
    print("Valores duplicados:")
    print(valores_duplicados)
else:
    print("Não existem pedidos duplicados na coluna ProtocoloPedido.")

Não existem pedidos duplicados na coluna ProtocoloPedido.


In [ ]:
# Verificando por pedidos duplicados na coluna 'DetalhamentoSolicitacao'
pedidos_duplicados = dt_pedidos['DetalhamentoSolicitacao'].duplicated()

# Verifica se há algum valor duplicado
tem_pedidos_duplicados = pedidos_duplicados.any()

if tem_pedidos_duplicados:
    print("Existem pedidos duplicados na coluna ProtocoloPedido.")
    # Opcional: para ver quais são os valores duplicados
    valores_duplicados = dt_pedidos['DetalhamentoSolicitacao'][pedidos_duplicados]
    print("Valores duplicados:")
    print(valores_duplicados)
else:
    print("Não existem pedidos duplicados na coluna ProtocoloPedido.")

Existem pedidos duplicados na coluna ProtocoloPedido.
Valores duplicados:
8         Prezados,  Gostaria de solicitar informações s...
335       Gostaria de receber o relatório final completo...
337       Gostaria de receber o relatório final completo...
397       Para avaliar as ações adotadas pela empresa no...
398       Para avaliar as ações adotadas pela empresa no...
                                ...                        
780068    Meu nome é Kalia Rivone Alves Barbosa e sou me...
780069    Meu nome é Kalia Rivone Alves Barbosa e sou me...
780070    Meu nome é Kalia Rivone Alves Barbosa e sou me...
780071    Meu nome é Kalia Rivone Alves Barbosa e sou me...
780073    Meu nome é Kalia Rivone Alves Barbosa e sou me...
Name: DetalhamentoSolicitacao, Length: 109813, dtype: object


In [ ]:
#Obs: podem existir recursos com o mesmo ProtocoloPedido, pois cada recurso se refere a uma instância, e assim um pedido pode ter vários recursos, caso um de uma instância

In [ ]:
# Verificar se os processos presentes nos dados rotulados estão presentes nos dados dos pedidos e recursos carregados

# Carregar dados rotulados
dt_rotulados = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/dataset_cgu/Base_anotada_NUPs_semelhantes.xlsx', sheet_name='NUP_semelhantes_limpos')

# obter números de protocolo
nup_pedidos_consulta = dt_rotulados['NUP'].unique()

# Listas para armazenar os números de protocolo presentes e ausentes
protocolos_presentes = []
protocolos_presentes_recursos = []
protocolos_ausentes = []


for numero in nup_pedidos_consulta:
  # Verifica se o número de protocolo está no DataFrame
  if numero in dt_pedidos['ProtocoloPedido'].values:
    protocolos_presentes.append(numero)
  else:
    if numero in dt_recursos['ProtocoloPedido'].values:
      protocolos_presentes_recursos.append(numero)
    else:
      protocolos_ausentes.append(numero)

# Imprime o total de processos presentes e ausentes
print(f"Total de processos presentes nos pedidos: {len(protocolos_presentes)}")
print(f"Total de processos ausentes nos pedidos mas presentes nos recursos: {len(protocolos_presentes_recursos)}")
print(f"Total de processos ausentes tanto em pedidos como em recursos: {len(protocolos_ausentes)}")

# Imprime a relação dos processos presentes nos pedidos
print("\nProcessos Presentes:")
for numero in protocolos_presentes:
  print(numero)

# Imprime a relação dos processos ausentes nos pedidos mas presentes nos recursos
print("\nProcessos Auentes nos Pedidos mas Presentes nos Recursos:")
for numero in protocolos_presentes_recursos:
  print(numero)

# Imprime a relação dos processos ausentes tanto em pedidos como em recursos
print("\nProcessos Ausentes tanto em Pedidos como Recursos:")
for numero in protocolos_ausentes:
  print(numero)

Total de processos presentes nos pedidos: 698
Total de processos ausentes nos pedidos mas presentes nos recursos: 0
Total de processos ausentes tanto em pedidos como em recursos: 405

Processos Presentes:
99945000797201961
99945000790201949
99945000742201870
99945000247201780
99923000959201919
99923000818202030
99923000668202064
99923000661202042
99923000660202006
99923000426202071
99923000415202091
99923000413202000
99922000665202031
99922000339201990
99920000219201911
99918000056201915
99916000020202087
99909002621201980
99909002501201982
99909002468202024
99909002299201999
99909002297201908
99909002296201955
99909002294201966
99909002293201911
99909002291201922
99908000419201923
99908000347201833
99902004970201653
99902003229201756
99902003226201631
99902001747201654
99902001131201764
99901001078202015
99901001071201951
99901000749202012
99901000708201992
99901000677201970
99901000648202041
99901000601202088
99901000591202081
99901000485202005
99901000098201846
99901000093201902
725

# 3 - Pipeline Recomendação

## Gerar Embeddings com SentenceTransformers

In [ ]:
# Baixar bibliotecas
!pip install -U -q sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.2 MB/s eta 0:00:00


In [ ]:
# Importar bibliotecas
from sentence_transformers import SentenceTransformer, util
import time
import torch
import faiss
import numpy as np

In [ ]:
#@title Carregar Modelo
# Carregar modelo sentence-transformers pré-treinado

model_pre = "intfloat/multilingual-e5-base" #@param ["all-MiniLM-L12-v2", "all-mpnet-base-v2", "sentence-t5-xxl", "multi-qa-MiniLM-L6-cos-v1", "msmarco-bert-base-dot-v5", "paraphrase-albert-small-v2", "all-roberta-large-v1", "msmarco-distilbert-base-v4", "paraphrase-multilingual-mpnet-base-v2", "roberta-base-nli-stsb-mean-tokens", "bert-base-nli-mean-tokens", "gtr-t5-large", "all-MiniLM-L6-v2", "paraphrase-MiniLM-L12-v2", "paraphrase-mpnet-base-v2","intfloat/e5-mistral-7b-instruct","intfloat/multilingual-e5-large","intfloat/multilingual-e5-base"]
model_st = SentenceTransformer(model_pre)

# Definir dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_st = model_st.to(device)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# Atributos que serão utilizados para gerar os embeddings

# Pedidos
dt_pedidos['sentence'] = dt_pedidos['ResumoSolicitacao'].fillna('') + ' <SEP> ' + dt_pedidos['DetalhamentoSolicitacao'].fillna('')

# Recursos
dt_recursos['sentence'] =  dt_recursos['TipoRecurso'].fillna('') + ' <SEP> ' + dt_recursos['DescRecurso'].fillna('')

Vetores para Pedidos

In [ ]:
# Gerar vetores dos Pedidos

start = time.time()

vetores_pedidos = model_st.encode(dt_pedidos['sentence'].astype(str), show_progress_bar = True, normalize_embeddings = True)

end = time.time()

elapsed_time = end - start
minutes = elapsed_time // 60
seconds = elapsed_time % 60
print(f'Tempo para processamento: {int(minutes)}m {int(seconds)}s \n Quantidade de vetores/pedidos: ', len(vetores_pedidos))

In [ ]:
# Salvar vetores em arquivo
# Formato CSV

start = time.time()

is_model_tuned = False
print('-> Salvando vetores em arquivo...')
dir_save_vectors = path + '/vetores/'
if not os.path.isdir(dir_save_vectors):
  os.makedirs(dir_save_vectors)

url_save_vectors = dir_save_vectors + str(model_pre) + '_ft_' + str(is_model_tuned) + '_vetores_pedidos_2015_2023.csv'

#Converter para DF e incluir ProtocoloPedido do registro na primeira coluna de cada linha
vetores_pedidos_df = pd.DataFrame(vetores_pedidos)
vetores_pedidos_df.insert(0,-1,dt_pedidos.ProtocoloPedido,True)
#salvar arquivo de vetores em um csv
vetores_pedidos_df.to_csv(url_save_vectors, header=False, index=False)

end = time.time()

print('-> Vetores salvos em:',url_save_vectors)
print('-> Tempo para salvar: %2.fs \n' % (end-start),'Quantidade de vetores: ', len(vetores_pedidos))

-> Salvando vetores em arquivo...
-> Vetores salvos em: /content/drive/MyDrive/Colab Notebooks/dataset_cgu/download_jan_2024/vetores/all-mpnet-base-v2_ft_False_vetores_pedidos_2015_2023.csv
-> Tempo para salvar: 590s 
 Quantidade de vetores:  780084


In [ ]:
# Salvar vetores em arquivo Pedido
# Formato pickle

start = time.time()

is_model_tuned = False
print('-> Salvando vetores em arquivo...')
dir_save_vectors = path +'/vetores/'
if not os.path.isdir(dir_save_vectors):
  os.makedirs(dir_save_vectors)

#salvar em pk
import pickle as pk
url_save_vectors = dir_save_vectors + str(model_pre) + '_ft_' + str(is_model_tuned) + '_vetores_pedidos_2015_2023.pk'
with open(url_save_vectors,'wb') as file:
  pk.dump(vetores_pedidos, file=file)

end = time.time()

print('-> Vetores salvos em:',url_save_vectors)
print('-> Tempo para salvar: %2.fs \n' % (end-start),'Quantidade de vetores: ', len(vetores_pedidos))

In [ ]:
# Carregar Vetores de Arquivo
# Formato CSV

is_model_tuned = False

from numpy import genfromtxt

print('-> Carregando vetores do arquivo')
start = time.time()
dir_save_vectors = path + '/vetores/'

try:
  url_save_vectors = dir_save_vectors + str(model_pre) + '_ft_' + str(is_model_tuned) + '_vetores_pedidos_2015_2023.csv'
  vetores_com_id = genfromtxt(url_save_vectors, delimiter=',',dtype='float32')
  vetores_pedidos = np.delete(vetores_com_id, 0, 1) # apagar a coluna com ID
  print('Vetores carregados com sucesso:')
  print('Vetores:',url_save_vectors)
  end = time.time()
  print('-> Tempo para carregamento: %2.fs \n' % (end-start),'Quantidade de vetores: ', len(vetores_pedidos))
except:
  print('Erro ao carregar vetores. Verifique se existe o arquivo para o dataset e modelo informado.')
  print('URL:',url_save_vectors)

-> Carregando vetores do arquivo
Vetores carregados com sucesso:
Vetores: /content/drive/MyDrive/Colab Notebooks/dataset_cgu/download_jan_2024/vetores/all-MiniLM-L12-v2_ft_False_vetores_pedidos_2015_2023.csv
-> Tempo para carregamento: 32s 
 Quantidade de vetores:  81268


In [ ]:
# Carregar Vetores de Arquivo Pedidos
# Formato pickle

is_model_tuned = False

from numpy import genfromtxt

print('-> Carregando vetores do arquivo')
start = time.time()
dir_save_vectors = path +'/vetores/'

try:
  import pickle as pk
  url_save_vectors = dir_save_vectors + str(model_pre) + '_ft_' + str(is_model_tuned) + '_vetores_pedidos_2015_2023.pk'
  with open(url_save_vectors,'rb') as file:
    vetores_pedidos = pk.load(file)

  print('Vetores carregados com sucesso:')
  print('Vetores:',url_save_vectors)
  end = time.time()
  print('-> Tempo para carregamento: %2.fs \n' % (end-start),'Quantidade de vetores: ', len(vetores_pedidos))
except:
  print('Erro ao carregar vetores. Verifique se existe o arquivo para o dataset e modelo informado.')
  print('URL:',url_save_vectors)

-> Carregando vetores do arquivo
Vetores carregados com sucesso:
Vetores: /content/drive/MyDrive/Colab Notebooks/dataset_cgu/download_jan_2024/vetores/intfloat/multilingual-e5-base_ft_False_vetores_pedidos_2015_2023.pk
-> Tempo para carregamento: 70s 
 Quantidade de vetores:  780084


Vetores para Recursos

In [ ]:
# Gerar vetores dos Recursos

start = time.time()

vetores_recursos = model_st.encode(dt_recursos['sentence'], show_progress_bar = True, normalize_embeddings = True)

end = time.time()

print('Tempo para processamento: %2.fs \n' % (end-start),'Quantidade de vetores/recursos: ', len(vetores_recursos))

Batches:   0%|          | 0/2567 [00:00<?, ?it/s]

Tempo para processamento: 245s 
 Quantidade de vetores/recursos:  82124


In [ ]:
# Alterar tipo de dados para utilizar no Faiss

#embeddings = np.array([vetor for vetor in vetores]).astype("float32")

Criar índices para Pedidos e Recursos

In [ ]:
### Criar índice para Pedidos

# Instanciar o indice
index_pedidos = faiss.IndexFlatIP(vetores_pedidos.shape[1])

# Passar o índice para um IndexIDMap
index_pedidos = faiss.IndexIDMap(index_pedidos)

# Adicionar vetores e Ids
#index.add_with_ids(vetores, dt_pedidos.IdPedido.values)
index_pedidos.add_with_ids(vetores_pedidos, dt_pedidos.ProtocoloPedido.values)

print(f"Número de vetores de pedidos incluídos no índice Faiss: {index_pedidos.ntotal}")

Número de vetores de pedidos incluídos no índice Faiss: 81268


In [ ]:
### Criar índice para Recursos

# Instanciar o indice
index_recursos = faiss.IndexFlatIP(vetores_recursos.shape[1])

# Passar o índice para um IndexIDMap
index_recursos = faiss.IndexIDMap(index_recursos)

# Adicionar vetores e Ids
#index.add_with_ids(vetores, dt_pedidos.IdPedido.values)
index_recursos.add_with_ids(vetores_recursos, dt_recursos.IdRecurso.values)

print(f"Número de vetores de recursos incluídos no índice Faiss: {index_recursos.ntotal}")

Número de vetores de recursos incluídos no índice Faiss: 82124


In [ ]:
# Salvar índice no disco
from faiss import write_index
write_index(index_pedidos, "index_pedidos.index")

In [ ]:
# Carregar índice do disco
from faiss import read_index
index_pedidos = read_index("index_pedidos.index")

In [ ]:
# Criar dicionario em separado para obter os vetores de pedidos através do número de protocolo
dic_protocolo_para_vetor_pedido = dict(zip(dt_pedidos['ProtocoloPedido'], vetores_pedidos))

In [ ]:
# Criar dicionario em separado para obter os vetores de recursos através do id
dic_id_para_vetor_recurso = dict(zip(dt_recursos['IdRecurso'], vetores_recursos))

Teste de busca de pedidos similares

In [ ]:
# Query de exeplo
dt_pedidos.iloc[5415]

IdPedido                                                             1704583
ProtocoloPedido                                            23480001623201615
Esfera                                                               Federal
OrgaoDestinatario                UFMG – Universidade Federal de Minas Gerais
Situacao                                                           Concluída
DataRegistro                                                      26/01/2016
ResumoSolicitacao                        Jornada de trabalho Ouvidoria e SIC
DetalhamentoSolicitacao    Gostaria de saber qual a jornada de trabalho d...
PrazoAtendimento                                                  15/02/2016
FoiProrrogado                                                            Não
FoiReencaminhado                                                         Não
FormaResposta                            Pelo sistema (com avisos por email)
OrigemSolicitacao                                                   Internet

In [ ]:
vetores_pedidos[5415]

array([ 5.74498847e-02,  2.07842560e-03, -1.66222802e-03, -2.70711258e-02,
        8.39850605e-02, -2.91486364e-02, -2.52750758e-02,  4.81269807e-02,
        5.29058985e-02,  5.78634143e-02, -6.41496778e-02,  2.08585896e-02,
       -4.79314998e-02,  6.72041476e-02,  2.71119121e-02,  2.55336147e-02,
        2.46007256e-02,  4.21427526e-02, -4.91427891e-02,  9.36578214e-03,
        1.27227738e-01,  3.13038118e-02, -3.22707370e-02, -1.47612402e-02,
       -1.09483050e-02, -9.33910906e-02,  2.23458130e-02,  2.34327279e-02,
       -3.71434074e-03, -4.98239184e-03, -2.45625880e-02, -2.56445706e-02,
        2.06809919e-02,  5.88466041e-02, -3.79713736e-02,  3.62957343e-02,
        4.74533252e-02, -8.67297687e-03,  6.36429712e-02,  6.27518669e-02,
       -1.54796271e-02, -4.09837477e-02, -4.42752652e-02, -6.44809604e-02,
       -4.66998704e-02, -6.65717050e-02,  5.49398065e-02,  3.99439558e-02,
        2.28924071e-03, -1.19987037e-02, -4.21852892e-04, -7.39532709e-03,
       -1.97669864e-02,  

In [ ]:
dic_protocolo_para_vetor_pedido[23480001623201615]

array([ 5.74498847e-02,  2.07842560e-03, -1.66222802e-03, -2.70711258e-02,
        8.39850605e-02, -2.91486364e-02, -2.52750758e-02,  4.81269807e-02,
        5.29058985e-02,  5.78634143e-02, -6.41496778e-02,  2.08585896e-02,
       -4.79314998e-02,  6.72041476e-02,  2.71119121e-02,  2.55336147e-02,
        2.46007256e-02,  4.21427526e-02, -4.91427891e-02,  9.36578214e-03,
        1.27227738e-01,  3.13038118e-02, -3.22707370e-02, -1.47612402e-02,
       -1.09483050e-02, -9.33910906e-02,  2.23458130e-02,  2.34327279e-02,
       -3.71434074e-03, -4.98239184e-03, -2.45625880e-02, -2.56445706e-02,
        2.06809919e-02,  5.88466041e-02, -3.79713736e-02,  3.62957343e-02,
        4.74533252e-02, -8.67297687e-03,  6.36429712e-02,  6.27518669e-02,
       -1.54796271e-02, -4.09837477e-02, -4.42752652e-02, -6.44809604e-02,
       -4.66998704e-02, -6.65717050e-02,  5.49398065e-02,  3.99439558e-02,
        2.28924071e-03, -1.19987037e-02, -4.21852892e-04, -7.39532709e-03,
       -1.97669864e-02,  

In [ ]:
# Buscar os top-k vizinhos / k = 10
D, I = index_pedidos.search(np.array([vetores_pedidos[5415]]), k=10)
print(f'Dot Product: {D.flatten().tolist()}\n\nIdPedido: {I.flatten().tolist()}')

Dot Product: [1.0000003576278687, 1.0000003576278687, 1.0000003576278687, 1.0000003576278687, 1.0000003576278687, 1.0000003576278687, 1.0000003576278687, 1.000000238418579, 1.000000238418579, 1.000000238418579]

IdPedido: [23480001623201615, 23480001630201617, 23480001629201692, 23480001616201613, 23480001631201661, 23480001621201626, 23480001589201689, 23480001546201601, 23480001553201603, 23480001545201659]


In [ ]:
# Buscar os top-k vizinhos / k = 10
D, I = index_pedidos.search(np.array([dic_protocolo_para_vetor_pedido[99902004970201653]]), k=10)
print(f'Dot Product: {D.flatten().tolist()}\n\nIdPedido: {I.flatten().tolist()}')

Dot Product: [0.9999999403953552, 0.9999999403953552, 0.9999998807907104, 0.9999998807907104, 0.961881160736084, 0.961881160736084, 0.9564037322998047, 0.9422494173049927, 0.9136298894882202, 0.910149335861206]

IdPedido: [99902002448201637, 99902003210201629, 99902004966201695, 99902004970201653, 99902003211201673, 99902004965201641, 99902005551201639, 99902003651201621, 99902003614201612, 99902003641201695]


In [ ]:
if I[0, 0] == 1217000743202221:
    # Criando um novo array sem o primeiro elemento da primeira linha
    I = I[:, 1:]

In [ ]:
I.flatten().tolist()

[3005179153202241,
 3005524897202224,
 3005074057202215,
 8198025688202204,
 23546036345202253,
 52021001900202243,
 60110001572202212,
 25072043578202251,
 25072041233202263]

In [ ]:
# Exibir os processo similares
df_similares = dt_pedidos[dt_pedidos['ProtocoloPedido'].isin(I.flatten().tolist())]

# Exibir o resultado
print(df_similares['DetalhamentoSolicitacao'])

8548                    Não consigo entra no site pra ver 
19357    As Forças Armadas são exceção na adotação do  ...
26899    meu boleto não aparece para pagar no site do I...
31311    O site do Ministério da Justiça está com um li...
32898    Boa tarde não estou conseguindo acessar a pagi...
47212    Boa noite! Gostaria de saber precisamente a qu...
57843    Onde consigo encontrar a Política Nacional de ...
61280    Bom Dia,    Gostaria de saber se há previsão d...
65495    Bom dia, solicito contato do responsável pelo ...
Name: DetalhamentoSolicitacao, dtype: object


In [ ]:
df_similares

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
5415,3980377,1217000743202221,Federal,CNPQ – Conselho Nacional de Desenvolvimento Ci...,Concluída,29/01/2022,O site do lattes vai aderir ao padrão gov.br ...,Pergunta 1 - O site do lattes vai aderir ao pa...,21/02/2022,Não,...,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,16/02/2022,"Prezado cidadão, Bom Dia! Informamos q...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
8548,4046895,3005074057202215,Federal,ME - Ministério da Economia,Concluída,15/02/2022,Problema,Não consigo entra no site pra ver,07/03/2022,Não,...,Pelo sistema (com avisos por email),Internet,5541761,Acesso à informação,,,18/02/2022,"Senhor(a), O Serviço de Informações ao Cida...",Acesso Negado,Pedido incompreensível
19357,4278859,3005179153202241,Federal,ME - Ministério da Economia,Concluída,11/04/2022,As Forças Armadas são exceção na adotação do ...,As Forças Armadas são exceção na adotação do ...,02/05/2022,Não,...,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,25/04/2022,"Senhor(a), O Serviço de Informações ao Cida...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
26899,4425927,23546036345202253,Federal,INEP – Instituto Nacional de Estudos e Pesquis...,Concluída,20/05/2022,boleto não aparece,meu boleto não aparece para pagar no site do I...,13/06/2022,Não,...,Pelo sistema (com avisos por email),Internet,0,Exame Nacional do Ensino Médio - Enem,,,03/06/2022,"Prezado(a) Senhor(a), Em atendimento ao pe...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
31311,4504682,60110001572202212,Federal,MJSP – Ministério da Justiça e Segurança Pública,Concluída,09/06/2022,Pedido de informação,O site do Ministério da Justiça está com um li...,04/07/2022,Sim,...,Pelo sistema (com avisos por email),Internet,2787772,Outros em Segurança e Ordem Pública,,,23/06/2022,"Senhora J.A.X.O.A., Em atenção ao seu pedid...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
32898,4534436,52021001900202243,Federal,BNDES – Banco Nacional de Desenvolvimento Econ...,Concluída,20/06/2022,acesso pagina,Boa tarde não estou conseguindo acessar a pagi...,11/07/2022,Não,...,Pelo sistema (com avisos por email),Internet,6073628,Acesso à informação,,Fale Conosco (Ouvidoria),21/06/2022,"Prezado(a) Senhor(a), Recebemos seu pedido ...",Acesso Concedido,Orientação sobre como encontrar a informação s...
47212,4800613,8198025688202204,Federal,PRF – Polícia Rodoviária Federal,Concluída,27/08/2022,Quantidade de vagas ociosas/em aberto,Boa noite! Gostaria de saber precisamente a qu...,29/09/2022,Sim,...,Pelo sistema (com avisos por email),Internet,5031138,Acesso à informação,,,28/09/2022,"Prezada Senhora Aline, Em resposta a sua so...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
57843,4985480,25072041233202263,Federal,MS – Ministério da Saúde,Concluída,22/10/2022,Política Nacional de Atenção Integral à Saúde ...,Onde consigo encontrar a Política Nacional de ...,16/11/2022,Não,...,Pelo sistema (com avisos por email),Internet,0,Outros em Saúde,,,25/10/2022,"Prezado (a) cidadão (a), Em resposta ao seu...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
61280,5043417,25072043578202251,Federal,ANVISA – Agência Nacional de Vigilância Sanitária,Concluída,10/11/2022,NaN,"Bom Dia, Gostaria de saber se há previsão d...",01/12/2022,Não,...,Pelo sistema (com avisos por email),Internet,6669512,Serviços e Sistemas,,,18/11/2022,"Prezado (a) Senhor(a), Com base nas infor...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
65495,5122083,3005524897202224,Federal,ME - Ministério da Economia,Concluída,06/12/2022,Informação sobre contato de anuncio,"Bom dia, solicito contato do responsável pelo ...",26/12/2022,Não,...,Pelo sist

## Gerar Embeddings com LLM

In [ ]:
# Instalar bibliotecas necessárias para utilizar LLMs
!pip install accelerate bitsandbytes einops chromadb langchain -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Carregar LLM

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModel
import torch
import torch.nn.functional as F
from torch import Tensor
import time

model_id = "intfloat/e5-mistral-7b-instruct"
#model_id = "HuggingFaceH4/zephyr-7b-beta"
#model_id = "mistralai/Mixtral-8x7B-v0.1"
#model_id = "mistralai/Mistral-7B-v0.1"

#Definir tamanho máximo da entrada do modelo
max_length = 4096

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True, load_in_4bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.28G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Definir funções para gerar os embeddings a partir do modelo

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def gerar_embeddings(input_texts,model,max_length):

  # Tokenize the input texts
  batch_dict = tokenizer(input_texts, max_length=max_length - 1, padding=True, truncation=True, return_tensors='pt')

  outputs = model(**batch_dict)
  embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

  # normalize embeddings
  embeddings = F.normalize(embeddings, p=2, dim=1)

  return embeddings

# deprecated
def last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

# deprecated
def generate_embeddings(input_texts,max_length):

  # Tokenize the input texts
  batch_dict = tokenizer(input_texts, max_length=max_length - 1, return_attention_mask=False, padding=False, truncation=True)

  # append eos_token_id to every input_ids
  #batch_dict['input_ids'] = [input_ids + [tokenizer.eos_token_id] for input_ids in batch_dict['input_ids']]

  batch_dict['input_ids'] = [input_ids + [tokenizer.eos_token_id] if isinstance(input_ids, list) else [input_ids, tokenizer.eos_token_id] for input_ids in batch_dict['input_ids']]

  batch_dict = tokenizer.pad(batch_dict, padding=True, return_attention_mask=True, return_tensors='pt')

  outputs = model(**batch_dict)
  embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

  # normalize embeddings
  embeddings = F.normalize(embeddings, p=2, dim=1)

  return embeddings

In [ ]:
## Gerar vetores e índice

In [ ]:
# Gerar vetores

start = time.time()

vetores = []

for pedido in dt_pedidos['DetalhamentoSolicitacao']:
  embedding = gerar_embeddings(pedido,model,max_length)
  vetores.append(embedding)

end = time.time()

print('Tempo para processamento: %2.fs \n' % (end-start),'Quantidade de vetores: ', len(vetores))

In [ ]:
lista = []
for pedido in dt_pedidos['DetalhamentoSolicitacao']:
  lista.append(pedido)

In [ ]:
len(lista)

69079

In [ ]:
# Instalar faiss
!pip install -U -q faiss-gpu

In [ ]:
### Criar índice

# Instanciar o indice
index = faiss.IndexFlatL2(vetores.shape[1])

# Passar o índice para um IndexIDMap
index = faiss.IndexIDMap(index)

# Adicionar vetores e Ids
index.add_with_ids(vetores, dt_pedidos.IdPedido.values)

print(f"Número de vetores incluídos no índice Faiss: {index.ntotal}")

In [ ]:
## Realizar busca

In [ ]:
# Query de exeplo
dt_pedidos.iloc[5415, 7]

'Pergunta 1 - O site do lattes vai aderir ao padrão  gov.br ? ]    Pergunta 2 - Porque no site do lattes não  .gov ?'

In [ ]:
query = gerar_embeddings(dt_pedidos.iloc[5415, 7],model,max_length)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


In [ ]:
query.shape

torch.Size([1, 4096])

## Realizar Experimentos - Dados Rotulados

### Pedidos

Carregar Dados Anotados e Filtrar somente os que constam no dataset para realizar a busca bem como gerar as métricas

In [ ]:
def carrega_dados_anotados():
  file_path = "/content/drive/MyDrive/Colab Notebooks/dataset_cgu/Base_anotada_NUPs_semelhantes.xlsx"
  df = pd.read_excel(file_path, sheet_name='NUP_semelhantes_limpos')
  df_selecionado = df.iloc[:, :2]
  return df_selecionado

def filtra_rotulos(df_rotulos, df_cgu):
  filtro = df_rotulos['NUP'].isin(df_cgu['ProtocoloPedido']) & df_rotulos['NUP_semelhante'].isin(df_cgu['ProtocoloPedido'])
  return df_rotulos[filtro]

In [ ]:
df_anotado_completo = carrega_dados_anotados()
df_anotado = filtra_rotulos(df_anotado_completo, dt_pedidos)

In [ ]:
len(df_anotado)

1113

In [ ]:
# obter números de protocolo
nup_pedidos_consulta = df_anotado['NUP'].unique().tolist()

In [ ]:
len(nup_pedidos_consulta)

638

Definir Funções

In [ ]:
# Função para obter um pedido através do NUP
def get_pedido(nup,df):
  registros_filtrados = df[df['ProtocoloPedido'] == nup]
  if not registros_filtrados.empty:
    return registros_filtrados.iloc[0]
  else:
    return 'null'

In [ ]:
# Função para buscar os pedidos similares a um pedido de consulta
# Retorna os scores seguidos dos IDs dos pedidos
def busca_pedidos_similares(nup,df,index,k):
  #pedido_consulta = get_pedido(nup,df)
  embeddings_pedido = dic_protocolo_para_vetor_pedido[nup]
  #embeddings_pedido = model_st.encode(df['DetalhamentoSolicitacao'], show_progress_bar = False, normalize_embeddings = True) # substituir por acesso direto ao vetor no indice
  D, I = index.search(np.array([embeddings_pedido]), k=k)
  # Remover do resultado o pedido de consulta caso esteja presente
  # Encontra a posição do nup em I
  posicao_nup = np.where(I == nup)[1]

  # Verifica se nup foi encontrado em I
  if posicao_nup.size > 0:
    posicao_nup = posicao_nup[0]  # Obtém a posição
    # Remove a ocorrência de nup de I e o elemento correspondente em D
    I = np.delete(I, posicao_nup, axis=1)
    D = np.delete(D, posicao_nup, axis=1)
  return D , I # scores , ids dos processo

In [ ]:
## Função que passa uma lista protocolos e o indice de pedidos e o argumento K
## retorna todos K pares similares para cada item da lista de protocolos
def simjoin_pedidos(protocolos,df,index,k):
  total_pares_similares = []
  for nup in protocolos:
    scores, protocolos_similares = busca_pedidos_similares(nup,df,index,k)
    for ps,score in zip(protocolos_similares.flatten().tolist(),scores.flatten().tolist()):
      total_pares_similares.append([nup,ps,score])
  return total_pares_similares

In [ ]:
# Realizar consulta dos pedidos similares para todos os protocolos em uma lista

# Definir K
k = 100

print("Realizando busca de similares para",len(nup_pedidos_consulta),"pedidos...")
print("k definido como: ",k)

start = time.time()

pares_pedidos_similares = simjoin_pedidos(nup_pedidos_consulta,dt_pedidos,index_pedidos,k+1)

end = time.time()

print('Tempo para processamento: %2.fs \n' % (end-start),'Quantidade pares de pedidos similares no resultado: ', len(pares_pedidos_similares))

Realizando busca de similares para 638 pedidos...
k definido como:  100
Tempo para processamento: 532s 
 Quantidade pares de pedidos similares no resultado:  63801


In [ ]:
pares_pedidos_similares

[[99902004970201653, 99902002448201637, 0.9999999403953552],
 [99902004970201653, 99902003210201629, 0.9999999403953552],
 [99902004970201653, 99902004966201695, 0.9999998807907104],
 [99902004970201653, 99902003211201673, 0.961881160736084],
 [99902004970201653, 99902004965201641, 0.961881160736084],
 [99902004970201653, 99902005551201639, 0.9564037322998047],
 [99902004970201653, 99902003651201621, 0.9422494173049927],
 [99902004970201653, 99902003614201612, 0.9136298894882202],
 [99902004970201653, 99902003641201695, 0.910149335861206],
 [99902004970201653, 99902003666201699, 0.9098097085952759],
 [99902003226201631, 99902001700201691, 0.8898816704750061],
 [99902003226201631, 99902000987201631, 0.8721337914466858],
 [99902003226201631, 99902000779201632, 0.8543656468391418],
 [99902003226201631, 99902001450201699, 0.8535065054893494],
 [99902003226201631, 99902003723201630, 0.853254497051239],
 [99902003226201631, 99902003281201621, 0.8484171628952026],
 [99902003226201631, 9990200

In [ ]:
# Converter a lista de resultados para dataframe
recomendacoes = pd.DataFrame(pares_pedidos_similares, columns=['pedido', 'pedido_similar', 'score'])

In [ ]:
len(pares_pedidos_similares)

63801

In [ ]:
# Gerar métricas com base nos dados rotulados

In [ ]:
df_anotado

,NUP,NUP_semelhante
69,99902004970201653,99902004966201695
72,99902003226201631,99902002876201660
76,99902001747201654,99902001745201665
1360,23480007218201619,23480004830201621
1556,9200000962201652,9200000420201680
1557,9200000962201652,9200000905201673
1558,9200000962201652,9200000904201629
1559,9200000962201652,9200000903201684
2099,77000907201651,77000808201679


In [ ]:
dt_pedidos[dt_pedidos['ProtocoloPedido'] == 99902003226201631]['DetalhamentoSolicitacao'].iloc[0]

'Solicito o numero de desligamentos de tecnico bancário da caixa econômica federal polo recife no período entre junho de 2014 e maio de 2016.'

In [ ]:
dt_pedidos[dt_pedidos['ProtocoloPedido'] == 99902002876201660]['DetalhamentoSolicitacao'].iloc[0]

'Boa noite,  Prezados venho por intermédio deste sistema de acesso a informação, regulado pela lei de nº 12.527, de 18 de novembro de 2011, solicitar o que se segue:   1- Quantitativo de terceirizados das empresas SANDES CONSERVAÇÃO E SERVIÇOS EIRELLI E SAAG - SERV DE ASSESSORIA E ADMINISTRAÇÃO LTDA - EPP Em ALAGOAS, que foram contratados para trabalhar na Caixa Econômica Federal no Estado de ALAGOAS entre os anos de 2014 e 2016.  2- – Quantos TÉCNICOS BANCÁRIOS NOVOS existiam em ALAGOAS em 2014, 2015 e quantos existem agora em 2016?  3- – Quantas mortes/aposentadorias/demissões/exonerações ocorreram para o cargo de TÉCNICO BANCÁRIO NOVO em ALAGOAS durante o período 17/06/2014 a 23/05/2016?  4- Quais as atribuições exercidas pelos terceirizados das empresas supracitadas?  5- – Quantas agências foram inauguradas durante o período 17/06/2014 a 23/05/2016 em ALAGOAS?  6- – Qual o número ideal de funcionários exercendo o cargo de TÉCNICO BANCÁRIO NOVO em cada agência? e qual o número ideal

In [ ]:
len(recomendacoes)

63801

#### Gerar Métricas

**Recall@k, Precisão@k e F1-Score@k**

Recall (Revocação ou Sensibilidade):

- A capacidade do sistema de encontrar todos os itens relevantes. No contexto do k específico, o recall@k mede quantos dos itens verdadeiramente relevantes (baseados nos dados rotulados) foram capturados nas top-k recomendações do sistema. Por exemplo, se existem 5 itens relevantes e o sistema acerta 3 deles nas top-k recomendações, o recall@k é 3/5.
Simplificando: "De todos os itens que são relevantes, quantos o sistema conseguiu recomendar nas top-k posições?"

Precisão:

A capacidade do sistema de recomendar apenas itens relevantes, minimizando as recomendações irrelevantes. A precisão@k mede quantos dos itens recomendados nas top-k posições são realmente relevantes. Se o sistema recomenda k itens e apenas alguns deles são verdadeiramente relevantes, a precisão@k será esse "alguns" dividido por k.
Simplificando: "Das top-k recomendações feitas pelo sistema, quantas são realmente relevantes?"

F1-Score:

Uma combinação harmônica de recall e precisão, fornecendo um único número que representa a eficácia do sistema ao equilibrar essas duas métricas.
O F1-score@k é calculado para cada k e considera tanto a precisão@k quanto o recall@k. Ele é útil quando você quer balancear a precisão e o recall, especialmente se eles têm pesos similares na importância do seu sistema.
Simplificando: "Como o sistema equilibra a capacidade de encontrar itens relevantes (recall) com a precisão de suas recomendações (precisão)?"

In [ ]:
k = 100

In [ ]:
##### DEPRECATED
#####

# Função para calcular as métricas recall, recall_macro, precisao e f1 no k especifico
from sklearn.metrics import recall_score

def calcula_metricas_at_k(recomendacoes, rotulos_verdadeiros, k):
  # recomendacoes: DataFrame com colunas ['pedido', 'pedido_similar', 'score']
  # rotulos_verdadeiros: DataFrame com colunas ['NUP', 'NUP_semelhante']
  # k: número de top recomendações a considerar

  recalls = []
  recalls_macro = []
  precisions = []
  f1_scores = []



  # Iterar sobre cada pedido nos dados de recomendacoes
  for nup_query in recomendacoes['pedido'].unique():

    # Obter os verdadeiros similares para o pedido nos dados rotulados
    verdadeiros_similares = set(rotulos_verdadeiros[rotulos_verdadeiros['NUP'] == nup_query]['NUP_semelhante'])

    # Obter os top-k similares recomendados pelo sistema
    top_k_recomendados = set(recomendacoes[recomendacoes['pedido'] == nup_query].nlargest(k, 'score')['pedido_similar'])

    # Calcular o número de verdadeiros positivos, falsos positivos e falsos negativos
    verdadeiros_positivos = len(verdadeiros_similares.intersection(top_k_recomendados))
    falsos_positivos = len(top_k_recomendados) - verdadeiros_positivos
    falsos_negativos = len(verdadeiros_similares) - verdadeiros_positivos

     # Atualizar o conjunto total de itens
    conjunto_total_itens = set()
    conjunto_total_itens.update(verdadeiros_similares)
    conjunto_total_itens.update(top_k_recomendados)
    total_itens = len(conjunto_total_itens)

    # Preparar vetores de verdadeiros e predições para usar com sklearn
    y_true = [1 if item in verdadeiros_similares else 0 for item in conjunto_total_itens]
    y_pred = [1 if item in top_k_recomendados else 0 for item in conjunto_total_itens]

    # Calcular recall e precisao para este pedido
    recall = verdadeiros_positivos / (verdadeiros_positivos + falsos_negativos) # recall considerando todos os verdadeiros_simlares
    precisao = verdadeiros_positivos / k

    # Calcular F1-score
    f1_score = 2 * (precisao * recall) / (precisao + recall) if (precisao + recall) > 0 else 0

    # Calcular recall usando sklearn com average='macro'
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    recalls_macro.append(recall_macro)

    recalls.append(recall)
    precisions.append(precisao)
    f1_scores.append(f1_score)

  # Calcular a média das métricas para todos os pedidos
  recall_medio_at_k = sum(recalls) / len(recalls) if recalls else 0
  precisao_media_at_k = sum(precisions) / len(precisions) if precisions else 0
  f1_score_medio_at_k = sum(f1_scores) / len(f1_scores) if f1_scores else 0
  recall_medio_macro_at_k = sum(recalls_macro) / len(recalls_macro)

  return recall_medio_at_k, recall_medio_macro_at_k, precisao_media_at_k, f1_score_medio_at_k

In [ ]:
# Função para calcular as métricas recall, recall_macro, precisao, f1 e map no k especifico
from sklearn.metrics import recall_score, precision_score, f1_score

def calcula_metricas_at_k(recomendacoes, rotulos_verdadeiros, k):
  # recomendacoes: DataFrame com colunas ['pedido', 'pedido_similar', 'score']
  # rotulos_verdadeiros: DataFrame com colunas ['NUP', 'NUP_semelhante']
  # k: número de top recomendações a considerar

  recalls = []
  recalls_macro = []
  precisions = []
  f1_scores = []

  # Iterar sobre cada pedido nos dados de recomendacoes
  for nup_query in recomendacoes['pedido'].unique():
    conjunto_total_itens = set()

    # Obter os verdadeiros similares para o pedido nos dados rotulados
    verdadeiros_similares = set(rotulos_verdadeiros[rotulos_verdadeiros['NUP'] == nup_query]['NUP_semelhante'])

    # Obter os top-k similares recomendados pelo sistema
    top_k_recomendados = set(recomendacoes[recomendacoes['pedido'] == nup_query].nlargest(k, 'score')['pedido_similar'])

    # Atualizar o conjunto total de itens
    conjunto_total_itens.update(verdadeiros_similares)
    conjunto_total_itens.update(top_k_recomendados)

    # Preparar vetores de verdadeiros e predições para usar com sklearn
    y_true = [1 if item in verdadeiros_similares else 0 for item in conjunto_total_itens]
    y_pred = [1 if item in top_k_recomendados else 0 for item in conjunto_total_itens]

    # Calcular recall, precisao e F1-score usando sklearn
    recall = recall_score(y_true, y_pred, zero_division=0)
    precisao = precision_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # Calcular recall usando sklearn com average='macro'
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)

    recalls.append(recall)
    recalls_macro.append(recall_macro)
    precisions.append(precisao)
    f1_scores.append(f1)

  # Calcular a média das métricas para todos os pedidos
  recall_medio_at_k = sum(recalls) / len(recalls) if recalls else 0
  precisao_media_at_k = sum(precisions) / len(precisions) if precisions else 0
  f1_score_medio_at_k = sum(f1_scores) / len(f1_scores) if f1_scores else 0

  recall_medio_macro_at_k = sum(recalls_macro) / len(recalls_macro)

  return recall_medio_at_k, recall_medio_macro_at_k, precisao_media_at_k, f1_score_medio_at_k

In [ ]:
# Calcular recall, precisão e f1-score para o k
recall_k,recall_macro_k,precisao_k,f1_score_k = calcula_metricas_at_k(recomendacoes,df_anotado,k)
print(f"Recall@{k}: {recall_k:.3f}")
print(f"Recall_macro@{k}: {recall_macro_k:.3f}")
print(f"Precisão@{k}: {precisao_k:.3f}")
print(f"F1-Score@{k}: {f1_score_k:.3f}")

Recall@100: 0.710
Recall_macro@100: 0.355
Precisão@100: 0.011
F1-Score@100: 0.021


In [ ]:
# variar o k e obter todas as metricas
start = time.time()
resultados_recall = {}
resultados_recall_macro = {}
resultados_precisao = {}
resultados_f1 = {}
for k in range(1, 101):
  recall_at_k,recall_macro_at_k,precisao_at_k,f1_at_k = calcula_metricas_at_k(recomendacoes, df_anotado, k)
  resultados_recall[k] = recall_at_k
  resultados_recall_macro[k] = recall_macro_at_k
  resultados_precisao[k] = precisao_at_k
  resultados_f1[k] = f1_at_k
  #print(f"Recall@{k}: {recall_at_k:.3f}")
  #print(f"Precisão@{k}: {precisao_at_k:.3f}")
  #print(f"F1-Score@{k}: {f1_at_k:.3f}")

# Criar DataFrame a partir dos resultados
df_resultados = pd.DataFrame({
    'k': list(resultados_recall.keys()),
    'Recall@k': list(resultados_recall.values()),
    'Recall_Macro@k': list(resultados_recall_macro.values()),
    'Precisão@k': list(resultados_precisao.values()),
    'F1-Score@k': list(resultados_f1.values()),
})

# Formatar as colunas para exibir apenas 3 casas decimais
df_resultados['Recall@k'] = df_resultados['Recall@k'].map('{:.3f}'.format)
df_resultados['Recall_Macro@k'] = df_resultados['Recall_Macro@k'].map('{:.3f}'.format)
df_resultados['Precisão@k'] = df_resultados['Precisão@k'].map('{:.3f}'.format)
df_resultados['F1-Score@k'] = df_resultados['F1-Score@k'].map('{:.3f}'.format)

end = time.time()

print('-> Tempo para gerar métricas: %2.fs \n' % (end-start))

# Exibir o DataFrame
print(df_resultados)

-> Tempo para gerar métricas: 355s 

      k Recall@k Recall_Macro@k Precisão@k F1-Score@k
0     1    0.168          0.154      0.210      0.179
1     2    0.254          0.130      0.158      0.187
2     3    0.306          0.153      0.130      0.174
3     4    0.345          0.173      0.111      0.160
4     5    0.365          0.182      0.096      0.145
..  ...      ...            ...        ...        ...
95   96    0.711          0.356      0.011      0.022
96   97    0.711          0.356      0.011      0.022
97   98    0.712          0.356      0.011      0.021
98   99    0.712          0.356      0.011      0.021
99  100    0.712          0.356      0.011      0.021

[100 rows x 5 columns]


In [ ]:
df_resultados

,k,Recall@k,Recall_Macro@k,Precisão@k,F1-Score@k
0,1,0.122,0.558,0.158,0.130
1,2,0.202,0.597,0.129,0.149
2,3,0.253,0.622,0.109,0.144
3,4,0.284,0.637,0.093,0.133
4,5,0.305,0.647,0.081,0.121
...,...,...,...,...,...
95,96,0.624,0.806,0.010,0.019
96,97,0.625,0.806,0.010,0.019
97,98,0.625,0.806,0.010,0.019
98,99,0.626,0.806,0.010,0.019


In [ ]:
# Salvar metricas em arquivo

start = time.time()

is_model_tuned = False
print('-> Salvando resultados em arquivo...')
dir_save_vectors = path + '/resultados/'
if not os.path.isdir(dir_save_vectors):
  os.makedirs(dir_save_vectors)

url_save_vectors = dir_save_vectors + str(model_pre) + '_ft_' + str(is_model_tuned) + '_resultados.csv'

#salvar arquivo de vetores em um csv
df_resultados.to_csv(url_save_vectors, header=True, index=False)

end = time.time()

print('-> Resultados salvos em:',url_save_vectors)
print('-> Tempo para salvar: %2.fs \n' % (end-start))

-> Salvando resultados em arquivo...
-> Resultados salvos em: /content/drive/MyDrive/Colab Notebooks/dataset_cgu/download_jan_2024/resultados/intfloat/multilingual-e5-base_ft_False_resultados.csv
-> Tempo para salvar:  0s 



**MAP (Mean Average Precision)**
A precisão média (AP) para um único processo é calculada da seguinte maneira:
- Para cada conjunto de recomendações até k (para k = 1 até k), calcule a precisão.
- Calcule a média dessas precisões.

O MAP é a média das precisões médias para todos os processos.

In [ ]:
# Função para calcular o MAP (Mean Average Precision)
def calcula_map(recomendacoes, rotulos_verdadeiros, k_max):
  # recomendacoes: DataFrame com colunas ['pedido', 'pedido_similar', 'score']
  # rotulos_verdadeiros: DataFrame com colunas ['NUP', 'NUP_semelhante']
  # k_max: número máximo de top recomendações a considerar

  average_precisions = []

  # Iterar sobre cada pedido nos dados de recomendacoes
  for nup_query in recomendacoes['pedido'].unique():
    # Obter os verdadeiros similares para o pedido nos dados rotulados
    verdadeiros_similares = set(rotulos_verdadeiros[rotulos_verdadeiros['NUP'] == nup_query]['NUP_semelhante'])
    total_verdadeiros_similares = len(verdadeiros_similares)

    if total_verdadeiros_similares == 0:
      continue

    # Obter os similares recomendados pelo sistema, ordenados por score
    recomendados_ordenados = recomendacoes[recomendacoes['pedido'] == nup_query].sort_values('score', ascending=False)['pedido_similar']

    precisions = []
    encontrados = 0

    # Calcular a precisão para cada k
    for i, recomendado in enumerate(recomendados_ordenados, start=1):
        if recomendado in verdadeiros_similares:
          encontrados += 1
          precisions.append(encontrados / i)

        if i == k_max:
          break

    # Calcular a precisão média (AP) para este pedido
    ap = sum(precisions) / min(total_verdadeiros_similares, k_max)
    average_precisions.append(ap)

  # Calcular o MAP (Mean Average Precision)
  map_score = sum(average_precisions) / len(average_precisions) if average_precisions else 0

  return map_score

In [ ]:
map = calcula_map(recomendacoes, df_anotado, k)
print("MAP: ",map)

MAP@k:  0.2207752685647817


In [ ]:
k=100

In [ ]:
# variar o k e obter o MAP a cada k
resultados_map = {}
for k in range(1, 101):
  map_at_k = calcula_map(recomendacoes, df_anotado, k)
  resultados_map[k] = map_at_k
  print(f"MAP@{k}: {map_at_k:.3f}")

MAP@1: 0.158
MAP@2: 0.174
MAP@3: 0.186
MAP@4: 0.190
MAP@5: 0.195
MAP@6: 0.199
MAP@7: 0.203
MAP@8: 0.205
MAP@9: 0.208
MAP@10: 0.209
MAP@11: 0.210
MAP@12: 0.212
MAP@13: 0.212
MAP@14: 0.213
MAP@15: 0.214
MAP@16: 0.214
MAP@17: 0.214
MAP@18: 0.214
MAP@19: 0.215
MAP@20: 0.215
MAP@21: 0.216
MAP@22: 0.216
MAP@23: 0.216
MAP@24: 0.216
MAP@25: 0.217
MAP@26: 0.217
MAP@27: 0.217
MAP@28: 0.218
MAP@29: 0.218
MAP@30: 0.218
MAP@31: 0.218
MAP@32: 0.218
MAP@33: 0.218
MAP@34: 0.218
MAP@35: 0.218
MAP@36: 0.218
MAP@37: 0.219
MAP@38: 0.219
MAP@39: 0.219
MAP@40: 0.219
MAP@41: 0.219
MAP@42: 0.219
MAP@43: 0.219
MAP@44: 0.219
MAP@45: 0.219
MAP@46: 0.219
MAP@47: 0.219
MAP@48: 0.219
MAP@49: 0.219
MAP@50: 0.219
MAP@51: 0.219
MAP@52: 0.219
MAP@53: 0.220
MAP@54: 0.220
MAP@55: 0.220
MAP@56: 0.220
MAP@57: 0.220
MAP@58: 0.220
MAP@59: 0.220
MAP@60: 0.220
MAP@61: 0.220
MAP@62: 0.220
MAP@63: 0.220
MAP@64: 0.220
MAP@65: 0.220
MAP@66: 0.220
MAP@67: 0.220
MAP@68: 0.220
MAP@69: 0.220
MAP@70: 0.220
MAP@71: 0.220
MAP@72: 0.220
M

**MRR (Mean Reciprocal Rank)**

De forma simplificada, o MRR responde à pergunta: "Quão rapidamente o sistema encontra a primeira resposta correta?"

passo a passo:

- Primeiro Item Relevante: Para cada consulta feita ao sistema (por exemplo, cada pesquisa ou recomendação), verifica a lista de respostas fornecidas pelo sistema e encontra a posição (ou "rank") do primeiro item que é considerado relevante ou correto.

- Recíproco do Rank: Para essa posição, você calcula o recíproco do rank. Por exemplo, se a primeira resposta correta está na posição 1 (ou seja, o sistema acertou de primeira), o recíproco é 1/1 = 1. Se a primeira resposta correta está na posição 2, o recíproco é 1/2 = 0.5, e assim por diante. Isso dá mais valor às respostas corretas que aparecem mais cedo na lista.

- Média dos Recíprocos: Depois de calcular o recíproco do rank para cada consulta, tira a média desses valores. Essa média é o MRR. Ela dá uma ideia geral de quão bem o sistema está performando em trazer a primeira resposta correta para o topo da lista, em média, em todas as consultas.

In [ ]:
def calcula_mrr(recomendacoes, rotulos_verdadeiros):
  # recomendacoes: DataFrame com colunas ['pedido', 'pedido_similar', 'score']
  # rotulos_verdadeiros: DataFrame com colunas ['NUP', 'NUP_semelhante']

  reciprocal_ranks = []

  # Iterar sobre cada pedido nos dados de recomendacoes
  for nup_query in recomendacoes['pedido'].unique():
    # Obter os verdadeiros similares para o pedido nos dados rotulados
    verdadeiros_similares = set(rotulos_verdadeiros[rotulos_verdadeiros['NUP'] == nup_query]['NUP_semelhante'])

    if not verdadeiros_similares:
      continue

    # Obter os similares recomendados pelo sistema, ordenados por score
    recomendados_ordenados = recomendacoes[recomendacoes['pedido'] == nup_query].sort_values('score', ascending=False)['pedido_similar']

    # Encontrar o rank do primeiro item relevante
    rank = next((i + 1 for i, recomendado in enumerate(recomendados_ordenados) if recomendado in verdadeiros_similares), None)

    # Calcular o recíproco do rank
    if rank is not None:
      reciprocal_ranks.append(1 / rank)

  # Calcular o MRR (Mean Reciprocal Rank)
  mrr_score = sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0

  return mrr_score

In [ ]:
mrr = calcula_mrr(recomendacoes, df_anotado)
print("MRR: ",mrr)

MRR:  0.39657101625040464


**NDCG (Normalized Discounted Cumulative Gain)**

 medida que ajuda a entender quão boas são as recomendações de um sistema, levando em consideração a posição (ou ordem) em que os itens relevantes aparecem. A ideia básica é que itens relevantes que aparecem mais cedo são mais valiosos do que itens relevantes que aparecem mais tarde. Passo para calcular:

- Classificar os Itens: a partir de uma lista de itens recomendados pelo sistema, cada um com um grau de relevância (quão útil é o item). A relevância é indicada como 0 (não relevante) ou 1 (relevante).

- Calcular o "Ganho": Para cada item na lista, calcular o "ganho" baseado na relevância. Itens mais relevantes dão um ganho maior. O ganho é descontado (reduzido) com base na posição do item. Um item relevante na primeira posição tem um ganho total maior do que um item relevante na segunda posição, e assim por diante.

- Soma Cumulativa: fazer uma soma cumulativa desses ganhos descontados ao longo da lista até um certo ponto k (por exemplo, os top-10 itens). Isso é chamado de DCG (Discounted Cumulative Gain).

- Normalizar o Ganho: O valor de DCG depende do número de itens relevantes. Para poder comparar o DCG entre consultas diferentes ou sistemas diferentes normalizar o DCG pelo melhor DCG possível (chamado de IDCG, que é o DCG que seria obtido se todos os itens relevantes estivessem no topo da lista, na ordem ideal). A divisão do DCG pelo IDCG dá o NDCG.

- NDCG: O valor de NDCG vai de 0 a 1. Um NDCG de 1 significa que os itens estão na ordem perfeita - todos os itens relevantes estão no topo na ordem correta. Quanto mais perto de 1, melhor a qualidade das recomendações em termos de ordenação de relevância.

In [ ]:
import numpy as np

def calcula_ndcg(recomendacoes, rotulos_verdadeiros, k):
  # recomendacoes: DataFrame com colunas ['pedido', 'pedido_similar', 'score']
  # rotulos_verdadeiros: DataFrame com colunas ['NUP', 'NUP_semelhante']
  # k: número de top recomendações a considerar

  ndcg_scores = []

  # Iterar sobre cada pedido nos dados de recomendacoes
  for nup_query in recomendacoes['pedido'].unique():
    verdadeiros_similares = set(rotulos_verdadeiros[rotulos_verdadeiros['NUP'] == nup_query]['NUP_semelhante'])

    # Obter os top-k similares recomendados pelo sistema
    top_k_recomendados = recomendacoes[recomendacoes['pedido'] == nup_query].nlargest(k, 'score')['pedido_similar']

    # Calcular DCG@k
    dcg_k = sum((1 / np.log2(i + 2)) if recomendado in verdadeiros_similares else 0 for i, recomendado in enumerate(top_k_recomendados))

    # Calcular IDCG@k
    idcg_k = sum((1 / np.log2(i + 2)) for i in range(min(len(verdadeiros_similares), k)))

    # Calcular NDCG@k
    ndcg_k = dcg_k / idcg_k if idcg_k > 0 else 0
    ndcg_scores.append(ndcg_k)

  # Calcular o NDCG médio sobre todos os pedidos
  mean_ndcg = sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0

  return mean_ndcg

In [ ]:
ndcg = calcula_ndcg(recomendacoes, df_anotado,k)
print(f"NDCG@{k}: {ndcg:.3f}")

NDCG@100: 0.318


In [ ]:
# variar o k e obter o NDCG a cada k
resultados_ndcg = {}
for k in range(1, 101):
  ndcg_at_k = calcula_ndcg(recomendacoes, df_anotado, k)
  resultados_ndcg[k] = ndcg_at_k
  print(f"NDCG@{k}: {ndcg_at_k:.3f}")

NDCG@1: 0.500
NDCG@2: 0.436
NDCG@3: 0.495
NDCG@4: 0.482
NDCG@5: 0.482
NDCG@6: 0.482
NDCG@7: 0.482
NDCG@8: 0.502
NDCG@9: 0.502
NDCG@10: 0.521


### Recursos

Definir funções

In [ ]:
# Função para obter os recursos vinculados a um NUP
def get_recursos(nup,df):
  registros_filtrados = df[df['ProtocoloPedido'] == nup]
  if not registros_filtrados.empty:
    return registros_filtrados
  else:
    return 'null'

In [ ]:
# Função para buscar os recursos similares a um recurso de consulta
# Retorna os scores seguidos dos IDs dos pedidos/recursos

def busca_recursos_similares(id_recurso,df,index,k):
  embeddings_recurso = dic_id_para_vetor_recurso[id_recurso]
  D, I = index.search(np.array([embeddings_recurso]), k=k)

  # Remover do resultado o pedido de consulta caso esteja presente
  # Encontra a posição do nup em I
  posicao_id = np.where(I == id_recurso)[1]
  # Verifica se id foi encontrado em I
  if posicao_id.size > 0:
    posicao_id = posicao_id[0]  # Obtém a posição
    # Remove a ocorrência de nup de I e o elemento correspondente em D
    I = np.delete(I, posicao_id, axis=1)
    D = np.delete(D, posicao_id, axis=1)
  return D , I # scores , ids dos recursos

In [ ]:
## Função que passa uma lista de recursos e o indice de recursos e o argumento K
## retorna todos K pares similares para cada item da lista de recursos
def simjoin_recursos(ids,df,index,k):
  total_pares_similares = []
  for id in ids:
    scores, ids_similares = busca_recursos_similares(id,df,index,k)
    for id_similar,score in zip(ids_similares.flatten().tolist(),scores.flatten().tolist()):
      total_pares_similares.append([id,id_similar,score])
  return total_pares_similares

In [ ]:
# Realizar consulta dos recursos similares para todos os recursos em uma lista

# Definir K
k = 10

print("Realizando busca de recursos similares...")
print("k definido como: ",k)

start = time.time()

pares_recursos_similares = simjoin_recursos([10006],dt_recursos,index_recursos,k+1)

end = time.time()

print('Tempo para processamento: %2.fs \n' % (end-start),'Quantidade pares de recursos similares no resultado: ', len(pares_pedidos_similares))

Realizando busca de recursos similares...
k definido como:  10
Tempo para processamento:  0s 
 Quantidade pares de recursos similares no resultado:  10


In [ ]:
# Query de exemplo
dt_recursos['DescRecurso'].iloc[0]

'Gostaria de ter acesso aos CONTRATOS do Programa Minha Casa Minha Vida, da Caixa Econômica Federal com as Empreiteiras (modalidade FAR) e da Caixa Economica Federal com as Entidades (Modalidade Entidades) no Faixa I município de São Paulo, desde 2009 até 2015.'

In [ ]:
pares_recursos_similares

[[10006, 56174, 0.7195524573326111],
 [10006, 23847, 0.7073958516120911],
 [10006, 27612, 0.6919189095497131],
 [10006, 45998, 0.6659243702888489],
 [10006, 14020, 0.6426013112068176],
 [10006, 26155, 0.6424277424812317],
 [10006, 47579, 0.6366281509399414],
 [10006, 13160, 0.6312045454978943],
 [10006, 6891, 0.6250792145729065],
 [10006, 24060, 0.6213263869285583]]

In [ ]:
recurso_similar = dt_recursos[dt_recursos['IdRecurso'] == 23847]
print(recurso_similar['DescRecurso'].iloc[0])

Gostaria de ter acesso tipologia/ metragem quadrada de cada uma das unidades habitacionais contratadas desde 2009 até 2015, para a faixa I do Programa Minha Casa Minha Vida na cidade de São Paulo.Seja para a modalidade FAR como Entidades FDS, incluindo o nome da empresa/ entidade responsável pelo empreendimento, endereço do empreendimento, e numero de unidades habitacionais contratadas. minha pergunta foi referente a metragem quadrada de cada uma das unidades habitacionais contratadas. 


# 4 - Funcionalidades Básicas - Fluxo da Solução

In [ ]:
## Fluxo da solução
# 1 - Entrada: Recurso
# 2 - Buscar pedidos similares ao pedido vinculado ao recurso
# 3 - Buscar recursos similares (que já possuem resposta) ao recurso de entrada
# 4 - Com base nos recursos similares, apresentar percentual que foram deferidos e não deferidos
# 5 - Com base no maior percentual (deferido ou indeferido) gerar texto de resposta passando exemplos do maior percentual como few shot

**1 - Entrada: Recurso de Exemplo para Busca**

In [ ]:
# recursos de 3ª instancia
dt_recursos_3instancia = dt_recursos[dt_recursos['Instancia'] == 'CGU']

In [ ]:
# 1 Entrada: Recurso de exemplo
#recurso_exemplo = dt_recursos.iloc[51299] # todos recursos
recurso_exemplo = dt_recursos_3instancia.iloc[1237] # somente 3 instancia
print(recurso_exemplo)
print(recurso_exemplo['DescRecurso'])

IdRecurso                                                          25993
IdRecursoPrecedente                                                24495
DescRecurso            Nenhum momento foi solicitado informações que ...
IdPedido                                                         1762778
IdSolicitante                                                    2651577
ProtocoloPedido                                        99901000511201619
OrgaoPedido                                     BB Tecnologia e Serviços
OrgaoDestinatario      CGU/SNAI - Secretaria Nacional de Acesso à Inf...
Instancia                                                            CGU
Situacao                                                      Respondido
DataRegistro                                                  30/05/2016
PrazoAtendimento                                              06/06/2016
OrigemSolicitacao                                               Internet
TipoRecurso            Justificativa para o sigilo 

**2 - Buscar pedidos similares ao pedido vinculado ao recurso**

In [ ]:
#Verificar pedido vinculado ao recurso
pedido_vinculado = dt_pedidos[dt_pedidos['ProtocoloPedido'] == recurso_exemplo['ProtocoloPedido']]

In [ ]:
pedido_vinculado

In [ ]:
#Buscar pedidos similares
k = 10
nup_pedido_vinculado = pedido_vinculado['ProtocoloPedido'].iloc[0]
pedidos_similares = simjoin_pedidos([nup_pedido_vinculado],dt_pedidos,index_pedidos,k+1)

In [ ]:
pedidos_similares

In [ ]:
#Exibir detalhamento da solicitações dos pedidos similares
i = 1
for pedido_similar in pedidos_similares:
  numero_protocolo = pedido_similar[1]
  registro = dt_pedidos[dt_pedidos['ProtocoloPedido'] == numero_protocolo]
  print(i,"-",registro['DetalhamentoSolicitacao'].iloc[0])
  i=i+1

**3 - Buscar recursos similares (que já possuem resposta) ao recurso de entrada**

In [ ]:
#Buscar recursos similares
k=10
recursos_similares = simjoin_recursos([recurso_exemplo['IdRecurso']],dt_recursos,index_recursos,k+1)

In [ ]:
recursos_similares

[[25993, 25997, 0.9999995827674866],
 [25993, 24495, 0.9999995827674866],
 [25993, 24490, 0.9999995827674866],
 [25993, 98819, 0.9494736790657043],
 [25993, 122579, 0.9463481307029724],
 [25993, 191845, 0.9459978342056274],
 [25993, 22940, 0.943631112575531],
 [25993, 22944, 0.943631112575531],
 [25993, 90531, 0.9434282779693604],
 [25993, 113693, 0.9425069689750671]]

In [ ]:
# Incluir no resultado somente os recursos que já possuem resultado final

lista_ids_recursos_similares_filtrados = []

for recurso_similar in recursos_similares:
  id_recurso = recurso_similar[1]
  registro_recurso = dt_recursos[dt_recursos['IdRecurso'] == id_recurso]
  if registro_recurso['Situacao'].iloc[0] != 'Em Tramitação':
    lista_ids_recursos_similares_filtrados.append(registro_recurso['IdRecurso'].iloc[0])

# Cria dataframe somente coms os registros com IdRecurso na lista_ids_recursos_similares_filtrados
dt_recursos_filtrados = dt_recursos[dt_recursos['IdRecurso'].isin(lista_ids_recursos_similares_filtrados)].copy()

# Cria uma coluna 'order' que indica a posição de cada IdRecurso na lista_ids
for i in dt_recursos_filtrados.index:
  id_recurso = dt_recursos_filtrados.loc[i, 'IdRecurso']
  dt_recursos_filtrados.loc[i, 'order'] = lista_ids_recursos_similares_filtrados.index(id_recurso)

# Ordena o DataFrame com base na coluna 'order'
dt_recursos_filtrados = dt_recursos_filtrados.sort_values('order')

In [ ]:
dt_recursos_filtrados

,IdRecurso,IdRecursoPrecedente,DescRecurso,IdPedido,IdSolicitante,ProtocoloPedido,OrgaoPedido,OrgaoDestinatario,Instancia,Situacao,DataRegistro,PrazoAtendimento,OrigemSolicitacao,TipoRecurso,DataResposta,RespostaRecurso,TipoResposta,sentence,order
9092,25997,24490,Nenhum momento foi solicitado informações que ...,1782413,2651577,99901000612201681,BB Tecnologia e Serviços,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,30/05/2016,06/06/2016,Internet,Justificativa para o sigilo insatisfatória/não...,22/07/2016,DECISÃO No exercício das atribuições a mi...,Indeferido,Justificativa para o sigilo insatisfatória/não...,0.0
8924,24495,22944,Nenhum momento foi solicitado informações que ...,1762778,2651577,99901000511201619,BB Tecnologia e Serviços,BB Tecnologia e Serviços,Segunda Instância,Respondido,23/05/2016,30/05/2016,Internet,Justificativa para o sigilo insatisfatória/não...,30/05/2016,"Prezado Cidadão, A empresa mantém a decisão ...",Deferido,Justificativa para o sigilo insatisfatória/não...,1.0
8923,24490,22940,Nenhum momento foi solicitado informações que ...,1782413,2651577,99901000612201681,BB Tecnologia e Serviços,BB Tecnologia e Serviços,Segunda Instância,Respondido,23/05/2016,30/05/2016,Internet,Justificativa para o sigilo insatisfatória/não...,30/05/2016,"Prezado Cidadão, A empresa mantém a decisã...",Deferido,Justificativa para o sigilo insatisfatória/não...,2.0
24175,98819,,Ocorre que as informações requeridas não são d...,1947969,2690092,3950001822201713,MF - Ministério da Fazenda,MF - Ministério da Fazenda,Primeira Instância,Respondido,05/07/2017,10/07/2017,Internet,Justificativa para o sigilo insatisfatória/não...,10/07/2017,"Prezados, Em reposta ao recurso interposto, i...",Indeferido,Justificativa para o sigilo insatisfatória/não...,3.0
3982,122579,,"Prezados, A informação solicitada não é sigil...",2085711,2270301,99902003999201537,CEF – Caixa Econômica Federal,CEF – Caixa Econômica Federal,Primeira Instância,Respondido,24/11/2015,30/11/2015,Internet,Justificativa para o sigilo insatisfatória/não...,27/11/2015,"Prezado (a) Cidadão (ã), 1.Conforme solici...",Indeferido,Justificativa para o sigilo insatisfatória/não...,4.0
81516,191845,,A informação é de interesse público e deve ser...,6492230,0,137018649202336,CC-PR – Casa Civil da Presidência da República,CC-PR – Casa Civil da Presidência da República,Primeira Instância,Respondido,01/12/2023,11/12/2023,Internet,Justificativa para o sigilo insatisfatória/não...,11/12/2023,"Prezado(a) Cidadão(ã), Em atenção ao recurso...",Indeferido,Justificativa para o sigilo insatisfatória/não...,5.0
8734,22940,,A justificativa apresentada pelo órgão vai con...,1782413,2651577,99901000612201681,BB Tecnologia e Serviços,BB Tecnologia e Serviços,Primeira Instância,Respondido,16/05/2016,23/05/2016,Internet,Justificativa para o sigilo insatisfatória/não...,23/05/2016,"Prezado Cidadão, A empresa mantém a decisã...",Deferido,Justificativa para o sigilo insatisfatória/não...,6.0
8735,22944,,A justificativa apresentada pelo órgão vai con...,1762778,2651577,99901000511201619,BB Tecnologia e Serviços,BB Tecnologia e Serviços,Primeira Instância,Respondido,16/05/2016,23/05/2016,Internet,Justificativa para o sigilo insatisfatória/não...,23/05/2016,"Prezado Cidadão, A empresa mantém a decisã...",Deferido,Justificativa para o sigilo insatisfatória/não...,7.0
43592,90531,89763,Ratificando a solicitação anterior. Destacar q...,1658432,2989642,99901000149201911,BB – Banco do Brasil S.A.,BB – Banco do Brasil S.A.,Segunda Instância,Respondido,13/03/2019,18/03/2019,Internet,Justificativa para o sigilo insatisfatória/não...,18/03/2019,"Prezado(a) Sr(a). Giordano, Segue em anexo re...",Não conhecimento,Justificativa para o sigilo insatisfatória/não...,8.0
2332,113693,,Não se trata de quebra de sigilo nenhum. Não f...,1969658,2401518,3950001811201571,IBGE – Fundação Instituto Brasileiro de Geogra...,IBGE – Fundação Instituto Brasileiro de Geogra...,Primeira Instância,Respondido,14/09/20

**4 - Com base nos recursos similares, apresentar percentual que foram deferidos e não deferidos**

In [ ]:
# Obtém a contagem de cada categoria na coluna TipoResposta
contagem_categorias = dt_recursos_filtrados['TipoResposta'].value_counts()

# Calcula os percentuais para cada categoria
percentual_categorias = (dt_recursos_filtrados['TipoResposta'].value_counts(normalize=True) * 100).round(2)

# Cria um DataFrame para mostrar os resultados
resultado = pd.DataFrame({
  'Quantidade': contagem_categorias,
  'Percentual (%)': percentual_categorias
})

# Exibe o resultado
print(resultado)

                  Quantidade  Percentual (%)
Indeferido                 5            50.0
Deferido                   4            40.0
Não conhecimento           1            10.0


**5 - Com base no maior percentual (deferido ou indeferido) gerar texto de resposta passando exemplos do maior percentual como few shot**

In [ ]:
# Instalar bibliotecas necessárias para utilizar LLMs
!pip install accelerate bitsandbytes einops chromadb langchain -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Carregar LLM

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "HuggingFaceH4/zephyr-7b-beta"
#model_id = "mistralai/Mixtral-8x7B-v0.1"
#model_id = "mistralai/Mistral-7B-v0.1"
#model_id = "recogna-nlp/bode-13b-alpaca-pt-br"
#model_id = "dominguesm/canarim-7b-vestibulaide"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, load_in_4bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Definir pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, return_full_text=False )

In [ ]:
# Decisão mais presente nos recursos similares
categoria_mais_comum = dt_recursos_filtrados['TipoResposta'].value_counts().idxmax()
print(categoria_mais_comum)

Indeferido


In [ ]:
# Texto da Instrução
instruction_fs = f"Você é um assistente para responder recursos dos pedidos de acesso à informação. Responda o recurso abaixo, considerando que o tipo da resposta será: {categoria_mais_comum}. Analise o texto do recurso e com base nas respostas de recursos similares anteriores, elabore a resposta com a devida justificativa da decisão."
instruction = f"Você é um assistente para responder recursos dos pedidos de acesso à informação. Responda o recurso abaixo, considerando que o tipo da resposta será: {categoria_mais_comum}. Analise o texto do recurso e elabore a resposta com a devida justificativa da decisão."

In [ ]:
## Geração de exemplos Few Shot a partir de pedidos similares

# Filtrar o DataFrame para apenas registros da catetoria mais comum
df_categoria_mais_comum = dt_recursos_filtrados[dt_recursos_filtrados['TipoResposta'] == categoria_mais_comum]

# Função para gerar os exemplos few-shot
def generate_few_shot(qtd_examples):

  exemplos_few_shot =  df_categoria_mais_comum.sample(n=qtd_examples)

  messages = []

  for index, row in exemplos_few_shot.iterrows():
    user_message = {"role": "user", "content": f"<tipo_recurso> {row['TipoRecurso']} </tipo_recurso> <recurso> {row['DescRecurso']} </recurso> <orgao> {row['OrgaoPedido']} </orgao> "}
    assistant_message = {"role": "assistant", "content": f"<resposta> {row['RespostaRecurso']} </resposta> <decisao> {row['TipoResposta']} </decisao>"}
    messages.append(user_message)
    messages.append(assistant_message)
  return messages

In [ ]:
df_categoria_mais_comum

,IdRecurso,IdRecursoPrecedente,DescRecurso,IdPedido,IdSolicitante,ProtocoloPedido,OrgaoPedido,OrgaoDestinatario,Instancia,Situacao,DataRegistro,PrazoAtendimento,OrigemSolicitacao,TipoRecurso,DataResposta,RespostaRecurso,TipoResposta,sentence,order
9092,25997,24490,Nenhum momento foi solicitado informações que ...,1782413,2651577,99901000612201681,BB Tecnologia e Serviços,CGU/SNAI - Secretaria Nacional de Acesso à Inf...,CGU,Respondido,30/05/2016,06/06/2016,Internet,Justificativa para o sigilo insatisfatória/não...,22/07/2016,DECISÃO No exercício das atribuições a mi...,Indeferido,Justificativa para o sigilo insatisfatória/não...,0.0
24175,98819,,Ocorre que as informações requeridas não são d...,1947969,2690092,3950001822201713,MF - Ministério da Fazenda,MF - Ministério da Fazenda,Primeira Instância,Respondido,05/07/2017,10/07/2017,Internet,Justificativa para o sigilo insatisfatória/não...,10/07/2017,"Prezados, Em reposta ao recurso interposto, i...",Indeferido,Justificativa para o sigilo insatisfatória/não...,3.0
3982,122579,,"Prezados, A informação solicitada não é sigil...",2085711,2270301,99902003999201537,CEF – Caixa Econômica Federal,CEF – Caixa Econômica Federal,Primeira Instância,Respondido,24/11/2015,30/11/2015,Internet,Justificativa para o sigilo insatisfatória/não...,27/11/2015,"Prezado (a) Cidadão (ã), 1.Conforme solici...",Indeferido,Justificativa para o sigilo insatisfatória/não...,4.0
81516,191845,,A informação é de interesse público e deve ser...,6492230,0,137018649202336,CC-PR – Casa Civil da Presidência da República,CC-PR – Casa Civil da Presidência da República,Primeira Instância,Respondido,01/12/2023,11/12/2023,Internet,Justificativa para o sigilo insatisfatória/não...,11/12/2023,"Prezado(a) Cidadão(ã), Em atenção ao recurso...",Indeferido,Justificativa para o sigilo insatisfatória/não...,5.0
2332,113693,,Não se trata de quebra de sigilo nenhum. Não f...,1969658,2401518,3950001811201571,IBGE – Fundação Instituto Brasileiro de Geogra...,IBGE – Fundação Instituto Brasileiro de Geogra...,Primeira Instância,Respondido,14/09/2015,21/09/2015,Internet,Justificativa para o sigilo insatisfatória/não...,13/10/2015,Considerando que o recurso não trouxe qualquer...,Indeferido,Justificativa para o sigilo insatisfatória/não...,9.0


Com Few-shot

In [ ]:
# Criar entradas para o prompt COM Few Shot

# Dados do recurso de entrada
tipo_recurso = recurso_exemplo['TipoRecurso']
descricao_recurso = recurso_exemplo['DescRecurso']
orgao_pedido = recurso_exemplo['OrgaoPedido']

messages = [{"role": "system", "content": instruction_fs}]

fs = generate_few_shot(2)

messages.extend(fs)

entrada_recurso = {"role": "user", "content": f"<tipo_recurso> {tipo_recurso} </tipo_recurso> <recurso> {descricao_recurso} </recurso> <orgao> {orgao_pedido} </orgao> "}

messages.append(entrada_recurso)

In [ ]:
messages

[{'role': 'system',
  'content': 'Você é um assistente para responder recursos dos pedidos de acesso à informação. Responda o recurso abaixo, considerando que o tipo da resposta será: Indeferido. Analise o texto do recurso e com base nas respostas de recursos similares anteriores, elabore a resposta com a devida justificativa da decisão.'},
 {'role': 'user',
  'content': '<tipo_recurso> Justificativa para o sigilo insatisfatória/não informada </tipo_recurso> <recurso> Ocorre que as informações requeridas não são de caráter pessoal, tendo em vista que os valores recebidos pelos servidores já são obrigatoriamente disponibilizados no portal da transparência. O que a entidade Sindical requer são apenas as informações funcionais do servidos abrangidos pelo seu estatuto. </recurso> <orgao> MF - Ministério da Fazenda </orgao> '},
 {'role': 'assistant',
  'content': '<resposta> Prezados,  Em reposta ao recurso interposto, informo que  o Portal da Transparência disponibiliza os dados funcionais

In [ ]:
# Inferência no LLM para gerar a reposta

start = time.time()

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
outputs = pipe(prompt, max_new_tokens=4000, do_sample=True, temperature=0.7, top_k=20, top_p=0.95)

end = time.time()

print('Tempo para gerar a reposta: %2.fs \n' % (end-start))

Tempo para gerar a reposta: 33s 



In [ ]:
# Exibir resposta COM Few-Shot
outputs[0]["generated_text"]

'<|assistant|>\n<resposta> Prezados,\n\n1. Conforme o solicitante não mencionou o momento específico solicitado, consideramos que se trata de informações de caráter geral, e, consequentemente, não podem estar sujeitas ao sigilo.\n\n2. De acordo com o Decreto nº 7.724/2012, que regula a Lei de Acesso à Informação - LAI, as informações que ofendam a privacidade e intimidade dos funcionários são protegidas e devem ser divulgadas apenas em cumprimento de um mandato legal ou com o consentimento expresso das pessoas a que se referem.\n\n3. Neste contexto, consideramos que as informações solicitadas estão presentes no Portal de Transparência do Governo Federal, e, portanto, podem ser acessadas sem restrições.\n\n4. Por outro lado, a negativa das informações fere o princípio da transparência da administração pública, que é fundamentada no Artigo 4º da Constituição Federal.\n\n5. Conforme disposto no parágrafo único do artigo 21 do Decreto 7.724/2012, o solicitante poderá apresentar um recurso 

Zero-shot

In [ ]:
# Criar entradas para o prompt Zero-shot

# Dados do recurso de entrada
tipo_recurso = recurso_exemplo['TipoRecurso']
descricao_recurso = recurso_exemplo['DescRecurso']
orgao_pedido = recurso_exemplo['OrgaoPedido']

messages = [{"role": "system", "content": instruction}]

entrada_recurso = {"role": "user", "content": f"<tipo_recurso> {tipo_recurso} </tipo_recurso> <recurso> {descricao_recurso} </recurso> <orgao> {orgao_pedido} </orgao> "}

messages.append(entrada_recurso)

In [ ]:
# Inferência no LLM para gerar a reposta

start = time.time()

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
outputs = pipe(prompt, max_new_tokens=4000, do_sample=True, temperature=0.7, top_k=20, top_p=0.95)

end = time.time()

print('Tempo para gerar a reposta: %2.fs \n' % (end-start))

Tempo para gerar a reposta: 34s 



In [ ]:
# Exibir resposta Zero-shot
outputs[0]["generated_text"]

'<|assistant|>\nO recurso solicitado pede a divulgação de informações que tratam do sigilo pessoal e intimidade dos funcionários da BB Tecnologia e Serviços. No entanto, a solicitação não fornece nenhum momento específico em que as informações foram requisitadas, o que dificulta a avaliação do risco para a privacidade e intimidade dos funcionários.\n\nEmbora a solicitação mencione que a informação solicitada esteja presente no portal de transparência do governo federal, isso não significa que todas as informações de sigilo e intimidade dos funcionários estejam disponíveis nesse portal. A divulgação de tais informações pode violar o princípio da transparência da administração pública, especialmente se for feita de maneira indiscriminada ou sem adevida justificativa.\n\nConsiderando o fato de que a solicitação não fornece um momento específico em que as informações foram requisitadas, e a possível violação do sigilo e intimidade dos funcionários, a decisão de não fornecer as informações 

In [ ]:
# Informações do recurso de exemplo:
print("Tipo do Recurso:",tipo_recurso)
print("Descrição:",descricao_recurso)
print("Órgão:",orgao_pedido)

Tipo do Recurso: Justificativa para o sigilo insatisfatória/não informada
Descrição: Nenhum momento foi solicitado informações que coloquem em risco a vida privada e/ou a intimidade dos funcionários. Esse tipo de informação que foi solicitada está presente no portal de transparência do governo federal.A negativa das informações fere o princípio da transparência da administração pública.
Órgão: BB Tecnologia e Serviços


Comparação com resposta dada pela CGU no recurso

In [ ]:
recurso_exemplo['TipoResposta']

'Indeferido'

In [ ]:
recurso_exemplo['RespostaRecurso']

'   DECISÃO     No exercício das atribuições a mim conferidas pela Portaria n. 1.567 da Controladoria-Geral da União, de 22 de agosto de 2013, adoto, como fundamento deste ato, o parecer acima, para decidir pelo desprovimento dos recursos interpostos, nostermos do art. 23 do referido Decreto, no âmbito dos pedidos de informação nº 99901.000612/2016-81 e 99901.000511/2016-19, direcionados à BB Tecnologia e Serviços.   GILBERTO WALLER JUNIOR Ouvidor-Geral da União   Nos termos do art. 24 do Decreto n° 7.724, V.Sa. poderá apresentar recurso à Comissão Mista de Reavaliação de Informações (CMRI), no prazo de 10 (dez) dias, contados da publicação da decisão do Ministério da Transparência, Fiscalização e Controle (antiga CGU). Nesse caso, deve-se clicar no botão correspondente, no sistema e-SIC, e apresentar as razões do recurso.  Conforme o disposto nos artigos 48 e 50 do Decreto 7.724/2012, a CMRI  se reunirá, ordinariamente, uma vez por mês  e deverá apreciar os recursos interpostos contra